# Задание 1 (3 балла)

Обучите word2vec модели с негативным семплированием (cbow и skip-gram) аналогично тому, как это было сделано в семинаре. Вам нужно изменить следующие пункты:
1) добавьте лемматизацию в предобработку (любым способом)  
2) измените размер окна в большую или меньшую сторону
3) измените размерность итоговых векторов

Выберете несколько не похожих по смыслу слов (не таких как в семинаре), и протестируйте полученные эмбединги (найдите ближайшие слова и оцените качество, как в семинаре).
Постарайтесь обучать модели как можно дольше и на как можно большем количестве данных. (Но если у вас мало времени или ресурсов, то допустимо взять поменьше данных и поставить меньше эпох)

### все install'ы

In [2]:
!pip install pymystem3

In [3]:
!wget https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/refs/heads/master/notebooks/word_embeddings/wiki_data.txt

--2025-12-27 13:56:32--  https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/refs/heads/master/notebooks/word_embeddings/wiki_data.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68582461 (65M) [text/plain]
Saving to: ‘wiki_data.txt’

wiki_data.txt       100%[===================>]  65.41M   209MB/s    in 0.3s    

2025-12-27 13:56:34 (209 MB/s) - ‘wiki_data.txt’ saved [68582461/68582461]



In [4]:
import re
from tqdm import tqdm
import numpy as np
# import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from pymystem3 import Mystem
from string import punctuation
from sklearn.model_selection import train_test_split
from collections import Counter
# import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.metrics.pairwise import cosine_distances

In [90]:
MyStem = Mystem(disambiguation=True)

In [6]:
wiki = open('wiki_data.txt').read().split('\n')

### предобработка

In [7]:
def lemmatize_sent(sent):
    res = MyStem.analyze(sent)
    lemma = []
    for analysis in res:
        lex = ''
        if 'analysis' in analysis and analysis['analysis']:
            lex = analysis['analysis'][0].get('lex', '')
        if lex: lemma.append(lex)
    return lemma

def preprocess(text):
    tokens = re.sub('#+', ' ', text.lower()).split()
    tokens = [token.strip(punctuation) for token in tokens]
    tokens = lemmatize_sent(' '.join(tokens))
    return tokens

In [8]:
print(wiki[0])
print(preprocess(wiki[0]))

######Новостройка (Нижегородская область)############Новостро́йка — сельский посёлок в Дивеевском районе Нижегородской области. Входит в состав Сатисского сельсовета.############Посёлок расположен в 12,5 км к югу от села Дивеева и 1 км к западу от города Сарова, на правом берегу реки Вичкинза (правый приток реки Сатис). Окружён смешанными лесами. Соединён асфальтовой дорогой с посёлком Цыгановка (1,5 км) и грунтовыми просёлочными дорогами с посёлком Сатис (3,5 км). Название Новостройка является сугубо официальным, местное население использует исключительно альтернативное название — Хитрый. Употребляется языковой оборот «…на Хитром». Ранее используемые названия — Песчаный, Известковый.############Основан в 1920-х годах переселенцами из соседних сёл Аламасово и Нарышкино (расположенных соответственно в 8 и 14 км к западу в Вознесенском районе).############Традиционно в посёлке жили рабочие совхоза «Вперёд» (центр в посёлке Сатис). Возле посёлка расположен карьер где активно добывали доло

In [9]:
# Предобработка всех данных
wiki_preprocessed = []
for sent in tqdm(wiki):
    processed = preprocess(sent)
    if processed:
        wiki_preprocessed.append(processed)

100%|██████████| 20003/20003 [04:10<00:00, 79.94it/s] 


In [10]:
word_counts = Counter()
for sent in wiki_preprocessed:
    word_counts.update(sent)

min_count = 10
vocab = [word for word, count in word_counts.items() if count >= min_count]

word2id = {'PAD': 0}
for idx, word in enumerate(vocab, 1):
    word2id[word] = idx
id2word = {idx: word for word, idx in word2id.items()}

print(f"Размер словаря: {len(vocab)}")
print(f"Примеры слов: {vocab[:20]}")

Размер словаря: 25849
Примеры слов: ['новостройка', 'нижегородский', 'область', 'сельский', 'поселок', 'в', 'дивеевский', 'район', 'входить', 'состав', 'сатисский', 'сельсовет', 'располагать', 'км', 'к', 'юг', 'от', 'садиться', 'дивеево', 'и']


In [11]:
sentences = []

for text in tqdm(wiki):
    tokens = preprocess(text)
    if not tokens:
        continue
    ids = [word2id[token] for token in tokens if token in word2id]
    sentences.append(ids)

100%|██████████| 20003/20003 [04:12<00:00, 79.29it/s] 


### модели

In [14]:
class CBOWNegSampling(nn.Module):
    def __init__(self, vocab_size, emb_dim, window_size):
        super().__init__()
        self.target_emb = nn.Embedding(vocab_size, emb_dim)
        self.context_emb = nn.Embedding(vocab_size, emb_dim)
        self.window_size = window_size

    def forward(self, target_ids, context_ids):
        """
        target_ids: (batch,)
        context_ids: (batch, window_size)
        """
        t = self.target_emb(target_ids)          # (batch, emb_dim)
        c = self.context_emb(context_ids)        # (batch, window, emb_dim)
        c_sum = c.sum(dim=1)                     # (batch, emb_dim)
        dot = (t * c_sum).sum(dim=1)             # (batch,)
        return dot

class SkipGramNegSampling(nn.Module):
    def __init__(self, vocab_size, emb_dim, window_size):
        super().__init__()
        self.center_emb = nn.Embedding(vocab_size, emb_dim)
        self.context_emb = nn.Embedding(vocab_size, emb_dim)
        self.window_size = window_size

    def forward(self, center_ids, context_ids):
        """
        center_ids: (batch,)
        context_ids: (batch,)
        """
        center = self.center_emb(center_ids)      # (batch, emb_dim)
        context = self.context_emb(context_ids)   # (batch, emb_dim)
        dot = (center * context).sum(dim=1)       # (batch,)
        return dot

In [17]:
def pad_sequences(sequences, maxlen, padding='post', value=0):
    """
    sequences: list of lists (разной длины)
    возвращает np.array shape (len(sequences), maxlen)
    """
    res = np.full((len(sequences), maxlen), value, dtype='int64')
    for i, seq in enumerate(sequences):
        if not seq:
            continue
        if len(seq) >= maxlen:
            if padding == 'post':
                res[i] = np.array(seq[:maxlen])
            else:
                res[i] = np.array(seq[-maxlen:])
        else:
            if padding == 'post':
                res[i, :len(seq)] = np.array(seq)
            else:
                res[i, -len(seq):] = np.array(seq)
    return res

In [27]:
def gen_batches_skipgram(sentences, window=5, batch_size=1000):
    left_context_length = (window/2).__ceil__()
    right_context_length = window // 2

    while True:
        X_center = []
        X_context = []
        y = []

        for sent in sentences:
            for i in range(len(sent)-1):
                center_word = sent[i]
                context_words = sent[max(0, i-left_context_length):i] + sent[i+1:i+right_context_length+1]

                for context_word in context_words:
                    X_center.append(center_word)
                    X_context.append(context_word)
                    y.append(1)

                    X_center.append(center_word)
                    X_context.append(np.random.randint(vocab_size))
                    y.append(0)

                if len(X_center) >= batch_size:
                    X_center_arr = np.array(X_center[:batch_size], dtype='int64')
                    X_context_arr = np.array(X_context[:batch_size], dtype='int64')
                    y_arr = np.array(y[:batch_size], dtype='float32')
                    yield (X_center_arr, X_context_arr), y_arr
                    X_center = X_center[batch_size:]
                    X_context = X_context[batch_size:]
                    y = y[batch_size:]

In [18]:
def gen_batches_cbow(sentences, window=5, batch_size=1000):
    left_context_length = (window/2).__ceil__()
    right_context_length = window // 2

    while True:
        X_target = []
        X_context = []
        y = []

        for sent in sentences:
            for i in range(len(sent)-1):
                word = sent[i]
                context = sent[max(0, i-left_context_length):i] + sent[i+1:i+right_context_length]

                X_target.append(word)
                X_context.append(context)
                y.append(1)

                X_target.append(np.random.randint(vocab_size))
                X_context.append(context)
                y.append(0)

                if len(X_target) >= batch_size:
                    X_target_arr = np.array(X_target, dtype='int64')
                    X_context_arr = pad_sequences(X_context, maxlen=window, padding='post', value=0)
                    y_arr = np.array(y, dtype='float32')
                    yield (X_target_arr, X_context_arr), y_arr
                    X_target, X_context, y = [], [], []

In [28]:
vocab_size = len(word2id)
emb_dim = 200
window_size = 7
num_epochs = 15
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [21]:
# CBOW
cbow_model = CBOWNegSampling(vocab_size, emb_dim, window_size).to(device)
optimizer_cbow = torch.optim.Adam(cbow_model.parameters(), lr=1e-3)

In [42]:
steps_per_epoch = 5000
validation_steps = 30
num_epochs = 10
criterion = nn.BCEWithLogitsLoss()
train_gen = gen_batches_cbow(sentences[:19000], window=window_size, batch_size=1000)
valid_gen = gen_batches_cbow(sentences[19000:], window=window_size, batch_size=1000)

for epoch in range(num_epochs):
    cbow_model.train()
    epoch_loss = 0.0
    for step in tqdm(range(steps_per_epoch)):
        (X_t, X_c), y = next(train_gen)
        X_t = torch.LongTensor(X_t).to(device)
        X_c = torch.LongTensor(X_c).to(device)
        y_t = torch.FloatTensor(y).to(device)

        optimizer_cbow.zero_grad()
        logits = cbow_model(X_t, X_c)
        loss = criterion(logits, y_t)
        loss.backward()
        optimizer_cbow.step()

        epoch_loss += loss.item()
    epoch_loss /= steps_per_epoch

    # validation
    cbow_model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for _ in range(validation_steps):
            (X_t, X_c), y = next(valid_gen)
            X_t = torch.LongTensor(X_t).to(device)
            X_c = torch.LongTensor(X_c).to(device)
            y_t = torch.FloatTensor(y).to(device)

            logits = cbow_model(X_t, X_c)
            loss = criterion(logits, y_t)
            val_loss += loss.item()
    val_loss /= validation_steps

    print(f"Epoch {epoch+1}/{num_epochs} - train loss: {epoch_loss:.4f}, val loss: {val_loss:.4f}")

100%|██████████| 5000/5000 [00:42<00:00, 117.05it/s]


Epoch 1/10 - train loss: 0.7342, val loss: 2.0102


100%|██████████| 5000/5000 [00:41<00:00, 120.05it/s]


Epoch 2/10 - train loss: 0.7099, val loss: 2.0100


100%|██████████| 5000/5000 [00:39<00:00, 125.19it/s]


...

Epoch 8/10 - train loss: 0.3317, val loss: 2.0018


100%|██████████| 5000/5000 [00:40<00:00, 124.65it/s]


Epoch 9/10 - train loss: 0.3013, val loss: 2.3357


100%|██████████| 5000/5000 [00:39<00:00, 125.52it/s]


Epoch 10/10 - train loss: 0.2871, val loss: 1.9685


In [44]:
embeddings = cbow_model.context_emb.weight.detach().cpu().numpy()

In [50]:
embeddings.shape

(25850, 200)

In [45]:
def most_similar(word, embeddings):
    similar = [id2word[i] for i in
               cosine_distances(embeddings[word2id[word]].reshape(1, -1), embeddings).argsort()[0][:10]]
    return similar

In [65]:
words = ['философия', 'болезнь', 'музыка', 'машина']

In [66]:
for word in words:
    print(f'Слово: {word}\n\t{most_similar(word, embeddings)}')

Слово: философия
	['философия', 'артур', 'религия', 'имеретинский', 'колесница', 'порождать', 'жак', 'ислам', 'рач', 'асановский']
Слово: болезнь
	['болезнь', 'заболевание', 'случай', 'выбывать', 'профессия', 'кен', 'препарат', 'экономичный', 'нищета', 'сестринский']
Слово: музыка
	['музыка', 'песня', 'сочинение', 'композитор', 'театральный', 'правота', 'тенор', 'концерт', 'театр', 'вибрация']
Слово: машина
	['машина', 'труднодоступный', 'размещение', 'увеличивать', 'надежность', 'колесо', 'курок', 'упорно', 'товар', 'мантия']


Кажется, что что-то обучилось даже:
- для философии: встречается религия, "артур" – мб Шопенгауэр, "жак" – Жан-Жак Руссо?
- для болезни: заболевание, нищета
- для музыки: песня, сочинение, композитор, тенор, концерт, театр, вибрации – все подходит
- для машины: если колесо... надежность и товар тоже осмысленно


In [68]:
# SkipGram
skip_model = SkipGramNegSampling(vocab_size, emb_dim, window_size).to(device)
optimizer_skip = torch.optim.Adam(skip_model.parameters(), lr=1e-3)

In [72]:
steps_per_epoch = 5000
validation_steps = 30
num_epochs = 20
criterion = nn.BCEWithLogitsLoss()
train_gen = gen_batches_skipgram(sentences[:19000], window=window_size, batch_size=1000)
valid_gen = gen_batches_skipgram(sentences[19000:], window=window_size, batch_size=1000)

for epoch in range(num_epochs):
    skip_model.train()
    epoch_loss = 0.0
    for step in tqdm(range(steps_per_epoch)):
        (X_t, X_c), y = next(train_gen)
        X_t = torch.LongTensor(X_t).to(device)
        X_c = torch.LongTensor(X_c).to(device)
        y_t = torch.FloatTensor(y).to(device)

        optimizer_skip.zero_grad()
        logits = skip_model(X_t, X_c)
        loss = criterion(logits, y_t)
        loss.backward()
        optimizer_skip.step()

        epoch_loss += loss.item()
    epoch_loss /= steps_per_epoch

    # validation
    skip_model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for _ in range(validation_steps):
            (X_t, X_c), y = next(valid_gen)
            X_t = torch.LongTensor(X_t).to(device)
            X_c = torch.LongTensor(X_c).to(device)
            y_t = torch.FloatTensor(y).to(device)

            logits = skip_model(X_t, X_c)
            loss = criterion(logits, y_t)
            val_loss += loss.item()
    val_loss /= validation_steps

    print(f"Epoch {epoch+1}/{num_epochs} - train loss: {epoch_loss:.4f}, val loss: {val_loss:.4f}")

100%|██████████| 5000/5000 [00:30<00:00, 164.04it/s]


Epoch 1/20 - train loss: 0.8883, val loss: 0.8414


100%|██████████| 5000/5000 [00:31<00:00, 159.24it/s]


Epoch 2/20 - train loss: 0.8357, val loss: 0.8929


100%|██████████| 5000/5000 [00:30<00:00, 164.23it/s]


...

Epoch 11/20 - train loss: 0.6722, val loss: 0.6163


100%|██████████| 5000/5000 [00:33<00:00, 147.54it/s]


Epoch 12/20 - train loss: 0.6107, val loss: 0.6584


  3%|▎         | 138/5000 [00:00<00:29, 167.44it/s]


KeyboardInterrupt: 

(обучила сначала на 10, потом еще на 10)

In [73]:
embeddings = skip_model.context_emb.weight.detach().cpu().numpy()

In [74]:
for word in words:
    print(f'Слово: {word}\n\t{most_similar(word, embeddings)}')

Слово: философия
	['философия', 'останки', 'медико', 'смешать', 'заведующий', 'бородкин', 'рацион', 'мао', 'следствие', 'оперный']
Слово: болезнь
	['болезнь', 'возможность', 'много', 'лист', 'использовать', 'со', 'из-за', 'второй', 'мочь', 'вода']
Слово: музыка
	['музыка', 'не', 'о', 'многий', 'работа', 'он', 'так', 'это', 'она', 'известный']
Слово: машина
	['машина', 'или', 'мочь', 'это', 'за', 'название', 'часть', 'иметь', 'однако', 'весь']


Со skipgram совсем все как-то неблизко вышло.

# Задание 2 (2 балла)

Обучите 1 word2vec и 1 fastext модель в gensim. В каждой из модели нужно задать все параметры, которые мы разбирали на семинаре. Заданные значения должны отличаться от дефолтных и от тех, что мы использовали на семинаре.

In [76]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 91.2 MB/s eta 0:00:00


In [77]:
import gensim

In [78]:
w2v = gensim.models.Word2Vec(
    wiki_preprocessed,
    vector_size=200,
    min_count=15,
    max_vocab_size=15000,
    window=8,
    epochs=15
)

In [80]:
for word in words:
    print(f'Слово: {word}\n\t{w2v.wv.most_similar(word)}')

Слово: философия
	[('социология', 0.6720197200775146), ('философский', 0.6267057061195374), ('математика', 0.621552586555481), ('литература', 0.6027094125747681), ('психология', 0.5849205851554871), ('лекция', 0.5761131644248962), ('религия', 0.5697950124740601), ('астрономия', 0.5573999881744385), ('докторский', 0.5480669736862183), ('монография', 0.5442533493041992)]
Слово: болезнь
	[('заболевание', 0.6812194585800171), ('туберкулез', 0.6477969288825989), ('лечение', 0.5960898995399475), ('приступ', 0.5943763256072998), ('страдать', 0.5897641777992249), ('сердечный', 0.5820220708847046), ('больной', 0.5781915187835693), ('боль', 0.5705823302268982), ('синдром', 0.5704296231269836), ('рак', 0.5518892407417297)]
Слово: музыка
	[('пение', 0.6459359526634216), ('композитор', 0.6363242268562317), ('музыкальный', 0.6213828325271606), ('фортепиано', 0.6014958024024963), ('мелодия', 0.5994106531143188), ('репертуар', 0.5950559973716736), ('джаз', 0.5773571133613586), ('композиция', 0.5731637

In [83]:
ft = gensim.models.FastText(
    wiki_preprocessed,
    vector_size=250,
    min_count=12,
    window=4,
    epochs=7,
    min_n=4,
    max_n=8
)

In [84]:
for word in words:
    print(f'Слово: {word}\n\t{ft.wv.most_similar(word)}')

Слово: философия
	[('философ', 0.958625316619873), ('филология', 0.8999954462051392), ('математика', 0.8433587551116943), ('политология', 0.8416385054588318), ('теология', 0.8344107866287231), ('социология', 0.8315575122833252), ('антология', 0.830829918384552), ('психология', 0.8301852345466614), ('софия', 0.8184142708778381), ('поэтика', 0.814580500125885)]
Слово: болезнь
	[('заболевание', 0.7950320243835449), ('заболеть', 0.7804254293441772), ('туберкулез', 0.7330014705657959), ('болейн', 0.6807439923286438), ('рак', 0.6555722951889038), ('боль', 0.6554720997810364), ('здоровье', 0.6397715210914612), ('болеть', 0.6372828483581543), ('симптом', 0.6367868781089783), ('заболевать', 0.6295412182807922)]
Слово: музыка
	[('поп-музыка', 0.9790514707565308), ('рок-музыка', 0.9784204363822937), ('музыкально', 0.9102877974510193), ('музыковед', 0.890175461769104), ('музыкант', 0.8644749522209167), ('мелодия', 0.7982892394065857), ('муза', 0.7873231172561646), ('джаз', 0.7746879458427429), ('к

# Задание 3 (3 балла)

Используя датасет для классификации (labeled.csv), обучите классификатор на базе эмбеддингов. Оцените качество на отложенной выборке.   
В качестве эмбеддинг модели вы можете использовать одну из моделей обученных в предыдущем задании или использовать одну из предобученных моделей с rusvectores (удостоверьтесь что правильно воспроизводите предобработку в этом случае!)  
Для того, чтобы построить эмбединг целого текста, усредните вектора отдельных слов в один общий вектор.
В качестве алгоритма классификации используйте LogisicticRegression (можете попробовать SGDClassifier, чтобы было побыстрее)  
F1 мера должна быть выше 20%.

In [85]:
!wget https://github.com/mannefedov/compling_nlp_hse_course/raw/refs/heads/master/notebooks/word_embeddings/labeled.csv

--2025-12-27 15:44:09--  https://github.com/mannefedov/compling_nlp_hse_course/raw/refs/heads/master/notebooks/word_embeddings/labeled.csv
Resolving github.com (github.com)... 140.82.116.4
Connecting to github.com (github.com)|140.82.116.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/refs/heads/master/notebooks/word_embeddings/labeled.csv [following]
--2025-12-27 15:44:10--  https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/refs/heads/master/notebooks/word_embeddings/labeled.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4669913 (4.5M) [application/octet-stream]
Saving to: ‘labeled.csv’

labeled.csv         100%[===================>]

In [86]:
import pandas as pd

In [87]:
data = pd.read_csv('labeled.csv')

In [88]:
data

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0
...,...,...
14407,Вонючий совковый скот прибежал и ноет. А вот и...,1.0
14408,А кого любить? Гоблина тупорылого что-ли? Или ...,1.0
14409,"Посмотрел Утомленных солнцем 2. И оказалось, ч...",0.0
14410,КРЫМОТРЕД НАРУШАЕТ ПРАВИЛА РАЗДЕЛА Т.К В НЕМ Н...,1.0


In [91]:
texts_preprocessed = []
for text in tqdm(data['comment']):
    tokens = preprocess(text)
    texts_preprocessed.append(tokens)

100%|██████████| 14412/14412 [00:24<00:00, 576.79it/s]


In [92]:
data['tokens'] = texts_preprocessed

In [94]:
data

,comment,toxic,tokens
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0,"[верблюд, то, за, что, дебил]"
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0,"[хохол, это, отдушина, затюканый, россиянин, м..."
2,Собаке - собачья смерть\n,1.0,"[собака, собачий, смерть]"
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0,"[страница, обновлять, дебил, это, тоже, не, ос..."
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0,"[ты, не, убеждать, страничный, в, то, что, скр..."
...,...,...,...
14407,Вонючий совковый скот прибежал и ноет. А вот и...,1.0,"[вонючий, совковый, скот, прибегать, и, ныть, ..."
14408,А кого любить? Гоблина тупорылого что-ли? Или ...,1.0,"[а, кто, любить, гоблин, тупорылый, что, ли, и..."
14409,"Посмотрел Утомленных солнцем 2. И оказалось, ч...",0.0,"[посмотреть, утомленный, солнце, и, оказыватьс..."
14410,КРЫМОТРЕД НАРУШАЕТ ПРАВИЛА РАЗДЕЛА Т.К В НЕМ Н...,1.0,"[крымотред, нарушать, правило, раздел, т, к, в..."


In [95]:
def get_text_embedding(tokens, model):
    embeddings = []
    for token in tokens:
        if token in model.wv:
            embeddings.append(model.wv[token])

    if len(embeddings) == 0:
        return np.zeros(model.wv.vector_size)

    return np.mean(embeddings, axis=0)

In [96]:
X_w2v = []
for tokens in tqdm(data['tokens']):
    emb = get_text_embedding(tokens, w2v)
    X_w2v.append(emb)

100%|██████████| 14412/14412 [00:01<00:00, 11285.80it/s]


In [99]:
X_w2v = np.array(X_w2v)
y = data['toxic'].values

In [100]:
X_train, X_test, y_train, y_test = train_test_split(
    X_w2v, y, test_size=0.2, random_state=42, stratify=y)

In [107]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, classification_report

In [102]:
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=42)

In [105]:
y_pred = model.predict(X_test)

In [138]:
f1 = f1_score(y_test, y_pred)
print(f"F1-score на тестовой выборке: {f1:.4f}")

F1-score на тестовой выборке: 0.6555


In [110]:
print(classification_report(y_test, y_pred, target_names=['non-toxic', 'toxic']))

              precision    recall  f1-score   support

   non-toxic       0.81      0.89      0.85      1918
       toxic       0.74      0.59      0.66       965

    accuracy                           0.79      2883
   macro avg       0.77      0.74      0.75      2883
weighted avg       0.79      0.79      0.79      2883



# Задание 4 (2 доп балла)

В тетрадку с фастекстом добавьте код для обучения с negative sampling (задача сводится к бинарной классификации) и обучите модель. Проверьте полученную модель на нескольких словах. Похожие слова должны быть похожими по смыслу и по форме.

In [111]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from tqdm import tqdm


In [112]:
import re
from collections import Counter
def tokenize(text):
    tokens = re.sub('#+', ' ', text.lower()).split()
    tokens = [token.strip(punctuation) for token in tokens]
    tokens = [token for token in tokens if token]
    return tokens

In [114]:
def ngrammer(raw_string, n=2):
    ngrams = []
    raw_string = ''.join(['<', raw_string, '>'])
    for i in range(0,len(raw_string)-n+1):
        ngram = ''.join(raw_string[i:i+n])
        if ngram == '<' or ngram == '>':
            continue
        ngrams.append(ngram)
    return ngrams

In [113]:
def split_tokens(tokens, min_ngram_size, max_ngram_size):
    tokens_with_subwords = []
    for token in tokens:
        subtokens = []
        for i in range(min_ngram_size, max_ngram_size+1):
            if len(token) > i:
                subtokens.extend(ngrammer(token, i))
        tokens_with_subwords.append(subtokens)
    return tokens_with_subwords

In [115]:
class SubwordTokenizer:
    def __init__(self, ngram_range=(1,1), min_count=5):
        self.min_ngram_size, self.max_ngram_size = ngram_range
        self.min_count = min_count
        self.subword_vocab = None
        self.fullword_vocab = None
        self.vocab = None
        self.id2word = None
        self.word2id = None

    def build_vocab(self, texts):
        # чтобы построить словарь нужно пройти по всему корпусу и собрать частоты всех уникальных слов и нграммов
        unfiltered_subword_vocab = Counter()
        unfiltered_fullword_vocab = Counter()
        for text in texts:
            tokens = tokenize(text)
            unfiltered_fullword_vocab.update(tokens)
            subwords_per_token = split_tokens(tokens, self.min_ngram_size, self.max_ngram_size)
            for subwords in subwords_per_token:
                # в одном слове могут быть одинаковые нграммы поэтому возьмем только уникальные
                unfiltered_subword_vocab.update(set(subwords))

        self.fullword_vocab = set()
        self.subword_vocab = set()

        # теперь отфильтруем по частоте
        for word, count in unfiltered_fullword_vocab.items():
            if count >= self.min_count:
                self.fullword_vocab.add(word)
        # для нграммов сделаем порог побольше чтобы не создавать слишком много нграммов
        # и учитывать только действительно частотные
        for word, count in unfiltered_subword_vocab.items():
            if count >= (self.min_count * 100):
                self.subword_vocab.add(word)

        # общий словарь
        self.vocab = self.fullword_vocab | self.subword_vocab
        self.id2word = {i:word for i,word in enumerate(self.vocab)}
        self.word2id = {word:i for i,word in self.id2word.items()}

    def subword_tokenize(self, text):
        if self.vocab is None:
            raise AttributeError('Vocabulary is not built!')
        # разбиваем на токены
        tokens = tokenize(text)
        # каждый токен разбиваем на символьные нграммы
        tokens_with_subwords = split_tokens(tokens, self.min_ngram_size, self.max_ngram_size)
        # оставляет только токены и нграммы которые есть в словаре
        only_vocab_tokens_with_subwords = []
        for full_token, sub_tokens in zip(tokens, tokens_with_subwords):
            filtered = []
            if full_token in self.vocab:
                # само слово и нграммы хранятся в одном списке
                # но слово будет всегда первым в списке
                filtered.append(full_token)
            filtered.extend([subtoken for subtoken in set(sub_tokens) if subtoken in self.vocab])
            only_vocab_tokens_with_subwords.append(filtered)

        return only_vocab_tokens_with_subwords

    def encode(self, subword_tokenized_text):
        # маппим токены и нграммы в их индексы в словаре
        encoded_text = []
        for token in subword_tokenized_text:
            if not token:
                continue
            encoded_text.append([self.word2id[token[0]]] + [self.word2id[t] for t in set(token[1:]) if t in self.word2id and t != token[0]])
        return encoded_text

    def __call__(self, text):
        return self.encode(self.subword_tokenize(text))

In [117]:
tokenizer = SubwordTokenizer(ngram_range=(2,4), min_count=10)
tokenizer.build_vocab(wiki)

In [118]:
def gen_batches_ft_neg_sampling(sentences, tokenizer, window=5, batch_size=1000,
                                 maxlen=20, num_negative=5):

    left_context_length = (window/2).__ceil__()
    right_context_length = window // 2
    vocab_size = len(tokenizer.fullword_vocab)

    while True:
        X = []
        y_context = []
        labels = []

        for sent in sentences:
            sent_encoded = tokenizer(sent)
            for i in range(len(sent_encoded)-1):
                word_with_subtokens = sent_encoded[i]
                context = sent_encoded[max(0, i-left_context_length):i] + \
                         sent_encoded[i+1:i+right_context_length+1]

                for context_word_with_subtokens in context:
                    only_full_word_context_token = context_word_with_subtokens[0]

                    X.append(word_with_subtokens)
                    y_context.append(only_full_word_context_token)
                    labels.append(1.0)

                    for _ in range(num_negative):
                        neg_context = np.random.randint(vocab_size)
                        X.append(word_with_subtokens)
                        y_context.append(neg_context)
                        labels.append(0.0)

                    if len(X) >= batch_size:
                        X_arr = pad_sequences(X[:batch_size], maxlen=maxlen, padding='post', value=0)
                        y_arr = np.array(y_context[:batch_size], dtype='int64')
                        labels_arr = np.array(labels[:batch_size], dtype='float32')
                        yield (X_arr, y_arr, labels_arr)
                        X = X[batch_size:]
                        y_context = y_context[batch_size:]
                        labels = labels[batch_size:]

In [119]:
class FastTextNegSampling(nn.Module):
    def __init__(self, vocab_size, emb_dim):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, emb_dim)

    def forward(self, X_center, X_context):
        center_embs = self.emb(X_center)  # (batch, maxlen, emb_dim)

        mask = (X_center != 0).float().unsqueeze(-1)  # (batch, maxlen, 1)
        center_embs = (center_embs * mask).sum(dim=1)  # (batch, emb_dim)
        count = mask.sum(dim=1).clamp(min=1)  # (batch, 1)
        center_embs = center_embs / count  # усредняем

        context_embs = self.emb(X_context)  # (batch, emb_dim)
        dot = (center_embs * context_embs).sum(dim=1)  # (batch,)

        return dot

In [122]:
device = "cuda" if torch.cuda.is_available() else "cpu"


vocab_size = len(tokenizer.vocab)
emb_dim = 100
window = 5
batch_size = 1000
num_negative = 5
num_epochs = 10
steps_per_epoch = 1000


model = FastTextNegSampling(vocab_size, emb_dim).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

train_gen = gen_batches_ft_neg_sampling(
    wiki[:19000], tokenizer,
    window=window,
    batch_size=batch_size,
    num_negative=num_negative
)
val_gen = gen_batches_ft_neg_sampling(
    wiki[19000:], tokenizer,
    window=window,
    batch_size=batch_size,
    num_negative=num_negative
)

In [123]:
train_losses = []
val_losses = []

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0.0

    for step in tqdm(range(steps_per_epoch), desc=f"Epoch {epoch+1}/{num_epochs}"):
        X, y_context, labels = next(train_gen)
        X = torch.LongTensor(X).to(device)
        y_context = torch.LongTensor(y_context).to(device)
        labels = torch.FloatTensor(labels).to(device)

        optimizer.zero_grad()
        scores = model(X, y_context)
        loss = criterion(scores, labels)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    epoch_loss /= steps_per_epoch
    train_losses.append(epoch_loss)

    model.eval()
    val_loss = 0.0
    validation_steps = 50

    with torch.no_grad():
        for step in range(validation_steps):
            X, y_context, labels = next(val_gen)
            X = torch.LongTensor(X).to(device)
            y_context = torch.LongTensor(y_context).to(device)
            labels = torch.FloatTensor(labels).to(device)

            scores = model(X, y_context)
            loss = criterion(scores, labels)
            val_loss += loss.item()

    val_loss /= validation_steps
    val_losses.append(val_loss)

    print(f"Epoch {epoch+1}/{num_epochs} - train loss: {epoch_loss:.4f}, val loss: {val_loss:.4f}")

Epoch 1/10: 100%|██████████| 1000/1000 [00:11<00:00, 90.23it/s]


Epoch 1/10 - train loss: 1.6687, val loss: 1.4187


Epoch 2/10: 100%|██████████| 1000/1000 [00:09<00:00, 106.71it/s]


Epoch 2/10 - train loss: 1.4156, val loss: 1.3345


Epoch 3/10: 100%|██████████| 1000/1000 [00:08<00:00, 114.95it/s]


...

Epoch 8/10 - train loss: 0.5265, val loss: 0.5051


Epoch 9/10: 100%|██████████| 1000/1000 [00:10<00:00, 92.72it/s]


Epoch 9/10 - train loss: 0.4704, val loss: 0.4790


Epoch 10/10: 100%|██████████| 1000/1000 [00:10<00:00, 94.48it/s]


Epoch 10/10 - train loss: 0.4460, val loss: 0.4246


In [124]:
from sklearn.metrics.pairwise import cosine_distances

embeddings = model.emb.weight.detach().cpu().numpy()

full_word_embeddings = np.zeros((len(tokenizer.fullword_vocab), emb_dim))
id2word = list(tokenizer.fullword_vocab)

for i, word in enumerate(tokenizer.fullword_vocab):
    subwords = tokenizer(word)[0]
    if subwords:
        full_word_embeddings[i] = embeddings[[idx for idx in subwords]].mean(axis=0)

In [134]:
def most_similar_ft(word, embeddings, tokenizer, topk=10):
    subwords = tokenizer(word)[0]
    word_embedding = embeddings[[i for i in subwords]].sum(axis=0)
    similar = [id2word[i] for i in
               cosine_distances(word_embedding.reshape(1, -1), full_word_embeddings).argsort()[0][:20]]
    return similar[:topk]

In [137]:
for word in words:
    print(f'Слово: {word}\n\t{most_similar_ft(word, embeddings, tokenizer, topk=10)}')

Слово: философия
	['философия', 'философию', 'философией', 'философы', 'философии', 'философ', 'философа', 'философов', 'философских', 'философии»']
Слово: болезнь
	['болезнь', 'болезнях', 'болезни»', 'болезни', 'болезнями', 'болезнью', 'болезней', 'полезно', 'болота', 'инновационный']
Слово: музыка
	['музыка', 'музыка»', 'музыку', 'музыкальные', 'музыкант', 'музыкальной', 'музыкальных', 'музыки»', 'музыкальном', 'музыке']
Слово: машина
	['машина', 'машинах', 'машин', 'машинами', 'машине', 'машиниста', 'машиностроение', 'машиностроения', 'машиной', 'части']


Больше акцент на форму получается, а если формы заканчивается, то "вылезают" абсолютно непонятные слова: "болезнь" –> "полезно" / "инновационный"